In [1]:
import json

# First, we read the dicts
deputies_dict = {}
for year in range(2001, 2022+1):

    with open(f'../backups/deputies_dict_backup/deputies_dict_{year}.json', 'r') as f:
        deputies_dict[year] = json.load(f)        

In [2]:
# Now we create a dict for the parties number of deputies and its proportion
parties_dict = {}
parties_prop_dict = {}
for year in range(2001, 2022+1):
    year_parties_dict = {}
    aux_dict = deputies_dict[year]
    for deputy in aux_dict:
        party = aux_dict[deputy]['party']
        if party not in year_parties_dict:
            year_parties_dict[party] = 1
        else:
            year_parties_dict[party] += 1
    
    parties_dict[year] = year_parties_dict
    total_deputies = 0
    for value in year_parties_dict.values():
        total_deputies += value
    parties_prop_dict[year] = {}
    for party in year_parties_dict:
        parties_prop_dict[year][party] = year_parties_dict[party] / total_deputies

In [3]:
# Plot the parties proportions evolution
import pandas as pd
import plotly.express as px

party_list = []
year_list = []
proportion_list = []
parties_agg_dict = {
    'PT': 'PT',
    'PSDB': 'PSDB',
    'PMDB': 'MDB',
    'MDB': 'MDB',
    'PSL': 'PSL',
    'PSOL': 'PSOL',
    'PL': 'PL',
    'PP': 'PP'
}
for year in range(2001, 2022+1):
    aux_dict = parties_prop_dict[year]  
    for party in aux_dict:
        if party in parties_agg_dict:
            party_list.append(parties_agg_dict[party])
        else:
            party_list.append('OUTROS')
        year_list.append(year)
        proportion_list.append(aux_dict[party])
             
color = {
    "MDB": 'Brown', 
    'PT': 'Crimson', 
    'PSDB': 'Blue', 
    'PSOL': 'Yellow',
    'PSL': 'Orange',
    'PL': 'Green',
    'PP': 'CornflowerBlue',
    'OUTROS': 'DarkGray'
}

parties_df = pd.DataFrame({'party': party_list, 'year': year_list, 'proportion': proportion_list})
parties_df = parties_df.groupby(['year', 'party']).agg({'proportion': 'sum'})
parties_df = parties_df.reset_index()
parties_df.head(10)
# parties_df = parties_df.sort_values(['party', 'year'])

,year,party,proportion
0,2001,MDB,0.165803
1,2001,OUTROS,0.507772
2,2001,PL,0.018135
3,2001,PSDB,0.189119
4,2001,PSL,0.002591
5,2001,PT,0.116580
6,2002,MDB,0.163339
7,2002,OUTROS,0.506352
8,2002,PL,0.023593
9,2002,PSDB,0.192377


In [4]:
# To show 'Outros' at the base of the plot
outros_idx = 1
sorted_idx = [outros_idx] + [i for i in range(len(parties_df)) if i != outros_idx]
parties_df = parties_df.iloc[sorted_idx]
parties_df.head(10)

,year,party,proportion
1,2001,OUTROS,0.507772
0,2001,MDB,0.165803
2,2001,PL,0.018135
3,2001,PSDB,0.189119
4,2001,PSL,0.002591
5,2001,PT,0.116580
6,2002,MDB,0.163339
7,2002,OUTROS,0.506352
8,2002,PL,0.023593
9,2002,PSDB,0.192377


In [5]:
fig = px.area(parties_df, x="year", y="proportion", color='party', line_group="party", color_discrete_map=color)
fig.update_layout(
    title="Proporção dos partidos por ano",
    xaxis_title="Ano",
    yaxis_title="Proporção",
    legend_title="Partidos",
    title_x=0.5,
    # font=dict(
    #     family="Courier New, monospace",
    #     size=18,
    #     color="RebeccaPurple"
    # )
)

fig.write_image('../imgs/parties_proportions.png')
fig.show()